In [ ]:
%pip install --upgrade --quiet pysqlite3 openai langchain langchain-community langchain-experimental

In [ ]:
#-------------
#import libraries
#-------------

from langchain_community.utilities.sql_database import SQLDatabase
import sqlite3
import pandas as pd
import os
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


#-------------
# Connect to quantum database
#-------------

conn = sqlite3.connect('quantum.db')
cursor = conn.cursor()

#-------------
#Create table 1
#-------------

query = '''
CREATE TABLE students_performance (
    StudentID INTEGER,
    Age INTEGER,
    Gender INTEGER,
    Ethnicity INTEGER,
    ParentalEducation INTEGER,
    StudyTimeWeekly DECIMAL(10),
    Absences INTEGER,
    Tutoring INTEGER,
    ParentalSupport INTEGER,
    Extracurricular INTEGER,
    Sports INTEGER,
    Music INTEGER,
    Volunteering INTEGER,
    GPA DECIMAL(10),
    GradeClass INTEGER

);

'''

cursor.execute(query)




In [ ]:

#-------------
#Read csv dataset 1
#-------------

df = pd.read_csv('/content/Student_performance_data _.csv',encoding='ISO-8859-1')
df.columns=['StudentID','Age','Gender','Ethnicity','ParentalEducation','StudyTimeWeekly','Absences','Tutoring','ParentalSupport','Extracurricular','Sports','Music','Volunteering','GPA','GradeClass']


#-------------
#Load the csv into database
#-------------

df.to_sql('students_performance', conn, if_exists='append', index=False)
conn.commit()

In [ ]:
#-------------
#Create table 2
#-------------

query = '''
CREATE TABLE popular_songs_2022 (
       Track varchar(255),
       Album_Name varchar(255),
       Artist varchar(255),
       ISRC varchar(255),
       All_Time_Rank integer,
       Track_Score decimal(10),
       Spotify_Streams integer ,
       Spotify_Playlist_Count integer,
       Spotify_Playlist_Reach integer,
       Spotify_Popularity integer,
       YouTube_Views integer,
       YouTube_Likes integer,
       TikTok_Posts integer,
       TikTok_Likes integer,
       TikTok_Views integer,
       YouTube_Playlist_Reach integer,
       Apple_Music_Playlist_Count integer,
       AirPlay_Spins integer,
       SiriusXM_Spins integer,
       Deezer_Playlist_Count integer,
       Deezer_Playlist_Reach integer,
       Amazon_Playlist_Count integer,
       Pandora_Streams integer,
       Pandora_Track_Stations integer,
       Soundcloud_Streams integer,
       Shazam_Counts integer,
       TIDAL_Popularity integer,
       Explicit_Track integer

);

'''

cursor.execute(query)





In [ ]:

#-------------
#Read csv dataset 2
#-------------

df2 = pd.read_csv('/content/spotify_songs_2022.csv',encoding = 'latin')
df2= df2[['Track', 'Album Name', 'Artist','ISRC',
       'All Time Rank', 'Track Score', 'Spotify Streams',
       'Spotify Playlist Count', 'Spotify Playlist Reach',
       'Spotify Popularity', 'YouTube Views', 'YouTube Likes', 'TikTok Posts',
       'TikTok Likes', 'TikTok Views', 'YouTube Playlist Reach',
       'Apple Music Playlist Count', 'AirPlay Spins', 'SiriusXM Spins',
       'Deezer Playlist Count', 'Deezer Playlist Reach',
       'Amazon Playlist Count', 'Pandora Streams', 'Pandora Track Stations',
       'Soundcloud Streams', 'Shazam Counts', 'TIDAL Popularity',
       'Explicit Track']]
df2.columns = ['Track', 'Album_Name', 'Artist', 'ISRC',
       'All_Time_Rank', 'Track_Score', 'Spotify_Streams',
       'Spotify_Playlist_Count', 'Spotify_Playlist_Reach',
       'Spotify_Popularity', 'YouTube_Views', 'YouTube_Likes', 'TikTok_Posts',
       'TikTok_Likes', 'TikTok_Views', 'YouTube_Playlist_Reach',
       'Apple_Music_Playlist_Count', 'AirPlay_Spins', 'SiriusXM_Spins',
       'Deezer_Playlist_Count', 'Deezer_Playlist_Reach',
       'Amazon_Playlist_Count', 'Pandora_Streams', 'Pandora_Track_Stations',
       'Soundcloud_Streams', 'Shazam_Counts', 'TIDAL_Popularity',
       'Explicit_Track']


#-------------
#Load the csv into database
#-------------

df2.to_sql('popular_songs_2022', conn, if_exists='append', index=False)
conn.commit()


In [ ]:
#-------------
# A Quick Check
#-------------

cursor.execute("SELECT * FROM students_performance limit 1")
print(cursor.fetchall())

[(1001, 17, 1, 0, 2, 19.833722807854716, 7, 1, 2, 0, 0, 1, 0, 2.929195591667681, 2)]


In [ ]:
#-------------
# OpenAI Key
#-------------

os.environ["OPENAI_API_KEY"] = "openai_key_goes_here"

In [ ]:

#-------------
# SQL Database Chaining + LLM
#-------------

db = SQLDatabase.from_uri("sqlite:///quantum.db")
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
#-------------
# Running the LLM
#-------------

db_chain.run("What's the average performance of the students?")



> Entering new SQLDatabaseChain chain...
What's the average performance of the students?
SQLQuery:SELECT AVG(GPA) FROM students_performance
SQLResult: [(1.9061863027265407,)]
Answer:The average performance of the students is 1.9061863027265407.
> Finished chain.


'The average performance of the students is 1.9061863027265407.'

In [ ]:
db_chain.run("What are the main characteristics of highly performing student?")



> Entering new SQLDatabaseChain chain...
What are the main characteristics of highly performing student?
SQLQuery:SELECT "StudentID", "Age", "Gender", "Ethnicity", "ParentalEducation", "StudyTimeWeekly", "Absences", "Tutoring", "ParentalSupport", "Extracurricular", "Sports", "Music", "Volunteering", "GPA", "GradeClass" FROM students_performance ORDER BY "GPA" DESC LIMIT 5
SQLResult: [(1045, 18, 1, 0, 1, 18.921512361376628, 1, 1, 3, 1, 1, 0, 0, 4, 0), (1443, 15, 0, 0, 2, 19.424398237709887, 0, 0, 3, 0, 1, 1, 1, 4, 0), (2279, 15, 1, 0, 2, 18.899695711116863, 3, 1, 3, 1, 1, 0, 0, 4, 0), (2706, 18, 1, 0, 2, 8.858281960839271, 0, 1, 4, 1, 0, 0, 0, 4, 0), (2920, 15, 0, 3, 1, 17.442121274931623, 1, 1, 1, 1, 1, 0, 0, 4, 0)]
Answer:The main characteristics of highly performing students are: being 15-18 years old, being male, having parents with a higher level of education, studying for around 18 hours per week, having low absences, receiving tutoring and parental support, participating in ext

'The main characteristics of highly performing students are: being 15-18 years old, being male, having parents with a higher level of education, studying for around 18 hours per week, having low absences, receiving tutoring and parental support, participating in extracurricular activities and sports, and having a high GPA and grade in class.'

In [ ]:
db_chain.run("What is the most popular song in 2022?")



> Entering new SQLDatabaseChain chain...
What is the most popular song in 2022?
SQLQuery:SELECT "Track" FROM popular_songs_2022 ORDER BY "Spotify_Streams" DESC LIMIT 1;
SQLResult: [('Best Day Of My Life',)]
Answer:Best Day Of My Life
> Finished chain.


'Best Day Of My Life'

In [ ]:
db_chain.run("How is the weather today?")



> Entering new SQLDatabaseChain chain...
How is the weather today?
SQLQuery:SELECT "Weather" FROM weather_forecast WHERE "Date" = date('now') LIMIT 1

OperationalError: (sqlite3.OperationalError) no such table: weather_forecast
[SQL: SELECT "Weather" FROM weather_forecast WHERE "Date" = date('now') LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)